<a href="https://colab.research.google.com/github/santro5048/Air-quality/blob/main/Code2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


# Install required libraries (run this cell first in Colab)
!pip install gradio scikit-learn pandas numpy

import gradio as gr
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

# Sample synthetic data generation function
def generate_sample_data(n_samples=1000, random_state=42):
    np.random.seed(random_state)
    data = pd.DataFrame({
        'PM2.5': np.random.uniform(5, 150, n_samples),
        'PM10': np.random.uniform(10, 200, n_samples),
        'NO2': np.random.uniform(5, 100, n_samples),
        'CO': np.random.uniform(0.2, 2.0, n_samples),
        'O3': np.random.uniform(10, 100, n_samples),
        'Temperature': np.random.uniform(10, 35, n_samples),
        'Humidity': np.random.uniform(20, 90, n_samples)
    })
    data['AQI'] = (0.4*data['PM2.5'] + 0.3*data['PM10'] + 0.1*data['NO2'] +
                   10*data['CO'] + 0.1*data['O3'] +
                   0.05*data['Temperature'] - 0.02*data['Humidity'] +
                   np.random.normal(0, 5, n_samples)).clip(0, 300)
    return data

# Function to classify AQI level
def classify_aqi(aqi):
    if aqi <= 50: return "Good"
    elif aqi <= 100: return "Moderate"
    elif aqi <= 150: return "Unhealthy for Sensitive Groups"
    elif aqi <= 200: return "Unhealthy"
    elif aqi <= 300: return "Very Unhealthy"
    else: return "Hazardous"

# Train model (run once)
print("Training model...")
data = generate_sample_data()
features = ['PM2.5', 'PM10', 'NO2', 'CO', 'O3', 'Temperature', 'Humidity']
X_train, X_test, y_train, y_test = train_test_split(
    data[features], data['AQI'], test_size=0.2, random_state=42
)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Gradio prediction function
def predict_aqi(pm25, pm10, no2, co, o3, temp, humidity):
    input_data = pd.DataFrame([[pm25, pm10, no2, co, o3, temp, humidity]],
                              columns=features)
    aqi = model.predict(input_data)[0]
    classification = classify_aqi(aqi)
    return f"Predicted AQI: {aqi:.1f}\nAir Quality: {classification}"

# Create Gradio interface
inputs = [
    gr.Slider(5, 150, label="PM2.5 (µg/m³)"),
    gr.Slider(10, 200, label="PM10 (µg/m³)"),
    gr.Slider(5, 100, label="NO2 (ppb)"),
    gr.Slider(0.2, 2.0, label="CO (ppm)"),
    gr.Slider(10, 100, label="O3 (ppb)"),
    gr.Slider(10, 35, label="Temperature (°C)"),
    gr.Slider(20, 90, label="Humidity (%)")
]

demo = gr.Interface(
    fn=predict_aqi,
    inputs=inputs,
    outputs="text",
    title="Air Quality Index Predictor",
    description="Predict AQI based on environmental parameters"
)

# Launch the Gradio app
print("Launching Gradio interface...")
demo.launch(share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
Training model...
Launching Gradio interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://38c1dd4854d71d79fa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
